# Move qubit along a line in the `Z` direction

This notebook aims at moving a logical qubit along a line during the logical memory experiment, 
which is depicted in the following figure:

![Move logical qubit along a line](./images/move_qubit_along_a_line/move_qubit_along_z.png)

To be more specific, we want to implement a distance $d$ logical memory experiment with the following steps:

1. Prepare the logical qubit in `Z` basis and repeat for $t_0 = d$ rounds
2. Extend the qubit along the line in the direction of `Z` logical operator and repeat for $t_1 = d$ rounds
3. Move the qubit to the destination and repeat for $t_2 = d$ rounds

The above steps will take $3d$ rounds in total. We will compare the logical `Z` error rate of the above experiment with the logical `Z` error rate of the standard static logical memory experiment. If the implementation is correct, the logical `Z` error rate of the above experiment should be at least as good as the logical `Z` error rate of the standard static logical memory experiment.

## Mandatory imports

In [ ]:
from copy import deepcopy

import cirq
import matplotlib.pyplot as plt
import sinter
import stim
from stimcirq import cirq_circuit_to_stim_circuit

from tqec.templates.constructions.qubit import (
    QubitRectangleTemplate,
    QubitSquareTemplate,
)
from tqec.templates.scale import Dimension, FixedDimension, LinearFunction
from tqec.templates.stack import StackedTemplate
from tqec.templates.shifted import ShiftedTemplate, ScalableOffset
from tqec.templates.atomic.rectangle import AlternatingRectangleTemplate
from tqec.detectors.transformer import transform_to_stimcirq_compatible
from tqec.enums import PlaquetteOrientation
from tqec.noise_models import (
    AfterCliffordDepolarizingNoise,
    AfterResetFlipNoise,
    BeforeMeasurementFlipNoise,
)
from tqec.plaquette.library import (
    XXFinalMeasurementPlaquette,
    XXInitialisationPlaquette,
    XXMemoryPlaquette,
    XXXXFinalMeasurementPlaquette,
    XXXXInitialisationPlaquette,
    XXXXMemoryPlaquette,
    ZZFinalMeasurementPlaquette,
    ZZInitialisationPlaquette,
    ZZMemoryPlaquette,
    ZZZZFinalMeasurementPlaquette,
    ZZZZInitialisationPlaquette,
    ZZZZMemoryPlaquette,
    ZZFromZZZZPlaquette,
)
from tqec.detectors.operation import make_observable
from tqec.position import Shape2D
from tqec.layers.base import BaseLayer
from tqec.display import display_template

## Normalisation and noisyness

Once the quantum error correction circuit implemented, we still need to apply two passes to obtain a circuit ready to be translated by the `stimcirq`.

The first pass normalises the `cirq.Circuit` produced. This pass was performing several modifications before, but is now simply removing potential empty `cirq.Moment` instances from the `cirq.Circuit` instance.

The second pass applies the noise model(s) we want to consider in the `stim` simulation.

In [ ]:
def normalise_circuit(circuit: cirq.Circuit) -> cirq.Circuit:
    ordered_transformers = [
        cirq.drop_empty_moments,
    ]
    for transformer in ordered_transformers:
        circuit = transformer(circuit)
    return circuit


def to_noisy_circuit(circuit: cirq.Circuit, noise_level: float) -> cirq.Circuit:
    noise_models = [
        AfterCliffordDepolarizingNoise(noise_level),
        AfterResetFlipNoise(noise_level),
        BeforeMeasurementFlipNoise(noise_level),
    ]
    for nm in noise_models:
        circuit = circuit.with_noise(nm)
    return circuit

## Build the templates

Firstly, we need to build templates for the plaquettes to fit in. Looking at the QEC image above, we will need 2 different templates.

In [ ]:
qubit_dimension = Dimension(2, LinearFunction(slope=2))
rectangle_width = Dimension(2, LinearFunction(slope=6))
square_template = QubitSquareTemplate(qubit_dimension)
rectangle_template = QubitRectangleTemplate(rectangle_width, qubit_dimension)
shifted_square_template = ShiftedTemplate(
    square_template,
    ScalableOffset(rectangle_width - qubit_dimension, FixedDimension(0)),
)
display_template(square_template)
print()
display_template(rectangle_template)
print()
display_template(shifted_square_template)

Due to implementation details, we also need several other templates that are obtained by stacking the two previously constructed ones and simple rectangles.

In [ ]:
rectangle_1xd_template = AlternatingRectangleTemplate(
    FixedDimension(1), qubit_dimension
)

movement_start_template = StackedTemplate()
movement_start_template.push_template_on_top(rectangle_template)
movement_start_template.push_template_on_top(square_template)
movement_start_template.push_template_on_top(
    ShiftedTemplate(
        rectangle_1xd_template,
        ScalableOffset(qubit_dimension + FixedDimension(1), FixedDimension(1)),
    )
)

movement_end_template = StackedTemplate()
movement_end_template.push_template_on_top(rectangle_template)
movement_end_template.push_template_on_top(shifted_square_template)
movement_end_template.push_template_on_top(
    ShiftedTemplate(
        rectangle_1xd_template,
        ScalableOffset(rectangle_width - qubit_dimension, FixedDimension(1)),
    )
)

display_template(movement_start_template)
print()
display_template(movement_end_template)

These stacked templates should be able to scale correctly:

In [ ]:
movement_start_template_copy = deepcopy(movement_start_template).scale_to(6)
movement_end_template_copy = deepcopy(movement_end_template).scale_to(6)
display_template(movement_start_template_copy)
print()
display_template(movement_end_template_copy)

## Prepare the layers

Layers are a new concept introduced recently in the `tqec` library. A layer is intended to represent one constant time-slice of a given QEC. In the QEC we are trying to implement in this notebook, we need to define several layers.

In the following cells, layers are defined as functions because we will need to generate these layers for different number of repetitions. They are not defined directly in the `generate_cirq_circuit_tqec` function below to allow interleaving code cells with textual explanations.

In [ ]:
from typing import Callable

# Ordered list that will store all the layers.
all_layers: list[Callable[[int], BaseLayer]] = []

### Logical qubit initialisation & memory

We first need to start the QEC by initialising a logical qubit.

In [ ]:
display_template(square_template)

In [ ]:
all_layers.append(
    lambda rep: BaseLayer(
        square_template,
        [
            # 1
            XXInitialisationPlaquette(
                PlaquetteOrientation.UP, [1, 2, 5, 6, 7, 8], include_detector=False
            ),
            ZZInitialisationPlaquette(PlaquetteOrientation.LEFT, [1, 5, 6, 8]),
            XXXXInitialisationPlaquette(
                [1, 2, 3, 4, 5, 6, 7, 8], include_detector=False
            ),
            ZZZZInitialisationPlaquette([1, 3, 4, 5, 6, 8]),
            # 5
            ZZInitialisationPlaquette(PlaquetteOrientation.RIGHT, [1, 3, 4, 8]),
            XXInitialisationPlaquette(
                PlaquetteOrientation.DOWN, [1, 2, 3, 4, 7, 8], include_detector=False
            ),
        ],
    )
)

We then need to perform a memory operation.

In [ ]:
all_layers.append(
    lambda rep: BaseLayer(
        square_template,
        [
            # 1
            XXMemoryPlaquette(PlaquetteOrientation.UP, [1, 2, 5, 6, 7, 8]),
            ZZMemoryPlaquette(PlaquetteOrientation.LEFT, [1, 5, 6, 8]),
            XXXXMemoryPlaquette([1, 2, 3, 4, 5, 6, 7, 8]),
            ZZZZMemoryPlaquette([1, 3, 4, 5, 6, 8]),
            # 5
            ZZMemoryPlaquette(PlaquetteOrientation.RIGHT, [1, 3, 4, 8]),
            XXMemoryPlaquette(PlaquetteOrientation.DOWN, [1, 2, 3, 4, 7, 8]),
        ],
        repetitions=rep,
    )
)

### Extension to the right & memory
After a few rounds of memory experiments, we want to extend the logical qubit to the right. The template will change to the rectangular template.
- Plaquettes that were measuring a 2-qubit stabiliser in the previous layer and that will measure a 4-qubit stabiliser in this layer do not have to be treated specifically as the operations and detectors should not change.
- Plaquettes that were already measuring a stabiliser should use a regular memory circuit.
- Plaquettes that were not initialised should use an initialisation circuit.

In [ ]:
# Print the template to be able to visualise correctly what we are doing
# and which plaquette goes where.
display_template(movement_start_template)

In [ ]:
all_layers.append(
    lambda rep: BaseLayer(
        movement_start_template,
        [
            # 1
            XXInitialisationPlaquette(
                PlaquetteOrientation.UP, [1, 2, 5, 6, 7, 8], include_detector=False
            ),
            # 2: not appearing in template because all instances are overriden by 8.
            ZZInitialisationPlaquette(PlaquetteOrientation.LEFT, [1, 5, 6, 8]),
            # TODO: resets two data qubits of 10.
            XXXXInitialisationPlaquette(
                [1, 2, 3, 4, 5, 6, 7, 8], include_detector=False
            ),
            ZZZZInitialisationPlaquette([1, 3, 4, 5, 6, 8]),
            # 5
            ZZInitialisationPlaquette(PlaquetteOrientation.RIGHT, [1, 3, 4, 8]),
            XXInitialisationPlaquette(
                PlaquetteOrientation.DOWN, [1, 2, 3, 4, 7, 8], include_detector=False
            ),
            # 7: beginning of the stacked template
            XXMemoryPlaquette(PlaquetteOrientation.UP, [1, 2, 5, 6, 7, 8]),
            ZZMemoryPlaquette(PlaquetteOrientation.LEFT, [1, 5, 6, 8]),
            XXXXMemoryPlaquette([1, 2, 3, 4, 5, 6, 7, 8]),
            # 10
            ZZZZMemoryPlaquette([1, 3, 4, 5, 6, 8]),
            # 11: was doing ZZ memory, should now do ZZZZ memory. No need to change the detector.
            ZZZZMemoryPlaquette([1, 3, 4, 5, 6, 8]),
            XXMemoryPlaquette(PlaquetteOrientation.DOWN, [1, 2, 3, 4, 7, 8]),
        ],
    )
)

Now that the logical qubit has been extended, we should perform some memory rounds on it.

In [ ]:
# Print the template to be able to visualise correctly what we are doing
# and which plaquette goes where.
display_template(rectangle_template)

In [ ]:
all_layers.append(
    lambda rep: BaseLayer(
        rectangle_template,
        [
            XXMemoryPlaquette(PlaquetteOrientation.UP, [1, 2, 5, 6, 7, 8]),
            ZZMemoryPlaquette(PlaquetteOrientation.LEFT, [1, 5, 6, 8]),
            XXXXMemoryPlaquette([1, 2, 3, 4, 5, 6, 7, 8]),
            ZZZZMemoryPlaquette([1, 3, 4, 5, 6, 8]),
            ZZMemoryPlaquette(PlaquetteOrientation.RIGHT, [1, 3, 4, 8]),
            XXMemoryPlaquette(PlaquetteOrientation.DOWN, [1, 2, 3, 4, 7, 8]),
        ],
        repetitions=rep,
    )
)

### Shrinking to the right
After a few rounds of memory experiments, we want to shrink the extended logical qubit to the right. The template will change back to the square template.

- Plaquettes that were measuring a 4-qubit stabiliser in the previous layer and that will measure a 2-qubit stabiliser in this layer have to be treated specifically: their detector should include the two measured qubits.
- Plaquettes that ends here should measure their data qubits.
- Other plaquettes should continue their memory experiment.

In [ ]:
# Print the template to be able to visualise correctly what we are doing
# and which plaquette goes where.
display_template(movement_end_template)

In [ ]:
all_layers.append(
    lambda rep: BaseLayer(
        movement_end_template,
        [
            # 1
            # TODO: measures one of the data qubits of 8.
            XXFinalMeasurementPlaquette(
                PlaquetteOrientation.UP, include_detector=False
            ),
            ZZFinalMeasurementPlaquette(PlaquetteOrientation.LEFT),
            # TODO: measures two of the data qubits of 10.
            XXXXFinalMeasurementPlaquette(include_detector=False),
            ZZZZFinalMeasurementPlaquette(),
            # 5: not appearing in template because all instances are overriden by 11.
            ZZFinalMeasurementPlaquette(PlaquetteOrientation.RIGHT),
            XXFinalMeasurementPlaquette(
                PlaquetteOrientation.DOWN, include_detector=False
            ),
            # 7: beginning of the stacked template
            XXMemoryPlaquette(PlaquetteOrientation.UP, [1, 2, 5, 6, 7, 8]),
            # 8: These were ZZZZ stabilizer measurements and becomes ZZ.
            ZZFromZZZZPlaquette(PlaquetteOrientation.LEFT, [1, 5, 6, 8]),
            XXXXMemoryPlaquette([1, 2, 3, 4, 5, 6, 7, 8]),
            # 10
            ZZZZMemoryPlaquette([1, 3, 4, 5, 6, 8]),
            ZZMemoryPlaquette(PlaquetteOrientation.RIGHT, [1, 3, 4, 8]),
            XXMemoryPlaquette(PlaquetteOrientation.DOWN, [1, 2, 3, 4, 7, 8]),
        ],
    )
)

### Shifted logical qubit memory & final measurements
We then need to perform a memory operation on the shifted logical qubit.

In [ ]:
# Print the template to be able to visualise correctly what we are doing
# and which plaquette goes where.
display_template(shifted_square_template)

In [ ]:
all_layers.append(
    lambda rep: BaseLayer(
        shifted_square_template,
        [
            # 1
            XXMemoryPlaquette(PlaquetteOrientation.UP, [1, 2, 5, 6, 7, 8]),
            ZZMemoryPlaquette(PlaquetteOrientation.LEFT, [1, 5, 6, 8]),
            XXXXMemoryPlaquette([1, 2, 3, 4, 5, 6, 7, 8]),
            ZZZZMemoryPlaquette([1, 3, 4, 5, 6, 8]),
            # 5
            ZZMemoryPlaquette(PlaquetteOrientation.RIGHT, [1, 3, 4, 8]),
            XXMemoryPlaquette(PlaquetteOrientation.DOWN, [1, 2, 3, 4, 7, 8]),
        ],
        repetitions=rep,
    )
)

Finally, we measure the logical qubit.

In [ ]:
all_layers.append(
    lambda rep: BaseLayer(
        shifted_square_template,
        [
            # 1
            XXFinalMeasurementPlaquette(
                PlaquetteOrientation.UP, include_detector=False
            ),
            ZZFinalMeasurementPlaquette(PlaquetteOrientation.LEFT),
            XXXXFinalMeasurementPlaquette(include_detector=False),
            ZZZZFinalMeasurementPlaquette(),
            # 5
            ZZFinalMeasurementPlaquette(PlaquetteOrientation.RIGHT),
            XXFinalMeasurementPlaquette(
                PlaquetteOrientation.DOWN, include_detector=False
            ),
        ],
    )
)

## Building the `cirq.Circuit` instance

Now that all layers have been defined, it becomes easy to build the final quantum circuit.

As a reminder, we want to implement
![Move logical qubit along a line](./images/move_qubit_along_a_line/move_qubit_along_z.png)

In [ ]:
def generate_cirq_circuit_tqec(k: int, repetitions: int) -> cirq.Circuit:
    # Create the cirq.Circuit instance by concatenating the circuits generated
    # for each layers and potentially modified by the modifiers defined above.
    circuit = cirq.Circuit()
    for layer_generator in all_layers:
        circuit += normalise_circuit(layer_generator(repetitions).generate_circuit(k))

    # Define the observable.
    # The observable is defined and added to the cirq.Circuit instance just here.
    # We assume that each plaquette has the same shape (i.e., needs the same number of qubits on the X and
    # Y dimensions). XX and ZZ stabilizers have been artificially made 3x3 plaquettes for this purpose. This
    # assumption will eventually need to be lifted.
    plaquette_shape: Shape2D = Shape2D(3, 3)
    origin = cirq.GridQubit(
        (1 + qubit_dimension.scale_to(k).value // 2) * (plaquette_shape.y - 1),
        plaquette_shape.x - 1,
    )
    observable_measurements = [
        (cirq.GridQubit(0, i * (plaquette_shape.x - 1)), -1)
        for i in range(rectangle_width.scale_to(k).value + 1)
    ]

    # circuit.append(cirq.Moment(make_observable(origin, observable_measurements)))

    # 6. Due to limitations in the API provided by Stim, the DetectorGate and ObservableGate instances
    # in the circuit only contain local measurement records, an internal representation that is not
    # understood by Stim. The fill_in_global_record_indices function replaces these local measurement
    # records by global ones that are understood by Stim.
    circuit_with_detectors = transform_to_stimcirq_compatible(circuit)
    return circuit_with_detectors

## Building the `stim.Circuit` instance

In [ ]:
def generate_stim_circuit_tqec(
    k: int, noise_level: float, repetitions: int
) -> stim.Circuit:
    circuit = generate_cirq_circuit_tqec(k, repetitions)
    if abs(noise_level) > 1e-12:
        circuit = to_noisy_circuit(circuit, noise_level)
    else:
        print(
            f"Warning: a noise level below 1e-12 ({noise_level}) has been detected. Noise is disabled. If this was intentional, please remove this code."
        )

    return cirq_circuit_to_stim_circuit(circuit)

In [ ]:
stim_circuit = generate_stim_circuit_tqec(2, 0, 10)
stim_circuit.explain_detector_error_model_errors()

In [ ]:
stim_circuit

## TQEC plots

We should finally be ready to perform the `stim` simulations and plot the results to check if our `tqec`-generated QEC memory experiment obtains the same results as the `stim`-generated one.

In [ ]:
surface_code_tasks_tqec = [
    sinter.Task(
        circuit=generate_stim_circuit_tqec((d - 1) // 2, noise, 3 * d),
        json_metadata={"d": d, "r": 3 * d, "p": noise},
    )
    for d in [3, 5, 7, 9, 11]
    for noise in [0.001, 0.002, 0.005, 0.009, 0.01, 0.011, 0.012, 0.013, 0.014]
]

collected_surface_code_stats_tqec: list[sinter.TaskStats] = sinter.collect(
    num_workers=20,
    tasks=surface_code_tasks_tqec,
    decoders=["pymatching"],
    max_shots=1_000_000,
    max_errors=5_000,
    print_progress=False,
)

In [ ]:
fig, ax = plt.subplots(1, 1)
sinter.plot_error_rate(
    ax=ax,
    stats=collected_surface_code_stats_tqec,
    x_func=lambda stat: stat.json_metadata["p"],
    group_func=lambda stat: stat.json_metadata["d"],
    failure_units_per_shot_func=lambda stat: stat.json_metadata["r"],
)
ax.loglog()
ax.set_title("TQEC: Surface Code Error Rates per Round under Circuit Noise")
ax.set_xlabel("Phyical Error Rate")
ax.set_ylabel("Logical Error Rate per Round")
ax.grid(which="major")
ax.grid(which="minor")
ax.legend()
fig.set_dpi(120)  # Show it bigger

## Stim plots

In [ ]:
surface_code_tasks_stim = [
    sinter.Task(
        circuit=generate_stim_circuit_stim(d, noise, 3 * d),
        json_metadata={"d": d, "r": 3 * d, "p": noise},
    )
    for d in [3, 5, 7, 9, 11]
    for noise in [0.001, 0.002, 0.005, 0.009, 0.01, 0.011, 0.012, 0.013, 0.014]
]

collected_surface_code_stats_stim: list[sinter.TaskStats] = sinter.collect(
    num_workers=20,
    tasks=surface_code_tasks_stim,
    decoders=["pymatching"],
    max_shots=1_000_000,
    max_errors=5_000,
    print_progress=False,
)

In [ ]:
fig, ax = plt.subplots(1, 1)
sinter.plot_error_rate(
    ax=ax,
    stats=collected_surface_code_stats_stim,
    x_func=lambda stat: stat.json_metadata["p"],
    group_func=lambda stat: stat.json_metadata["d"],
    failure_units_per_shot_func=lambda stat: stat.json_metadata["r"],
)
ax.loglog()
ax.set_title("Stim: Surface Code Error Rates per Round under Circuit Noise")
ax.set_xlabel("Phyical Error Rate")
ax.set_ylabel("Logical Error Rate per Round")
ax.grid(which="major")
ax.grid(which="minor")
ax.legend()
fig.set_dpi(120)  # Show it bigger

## Circuit visualisation

Some debugging relicates that are left here if you want to visualise/compare the quantum circuits generated.

In [ ]:
stim_circuit_tqec = generate_stim_circuit_tqec(1, 0.001, 2)

In [ ]:
stim_circuit_tqec.diagram("timeslice-svg")